In [1]:
import numpy as np
import tensorflow as tf

import scipy
import os
from os import listdir
from scipy.io import loadmat

In [2]:
from tensorflow.python.client import device_lib


from tensorflow.compat.v1.keras.backend import set_session

config = tf.compat.v1.ConfigProto()

config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU

config.log_device_placement = True  # to log device placement (on which device the operation ran)

sess = tf.compat.v1.Session(config=config)

set_session(sess)
print(device_lib.list_local_devices())


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 210820527211751965
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 201488586902541489
physical_device_desc: "device: XLA_CPU device"
]


In [3]:
directory = "./data/eeg_feature_smooth/1/"
directories = ["./data/eeg_feature_smooth/{}/".format(i+1) for i in range(3)] 
print(directories)


['./data/eeg_feature_smooth/1/', './data/eeg_feature_smooth/2/', './data/eeg_feature_smooth/3/']


In [4]:
n = 24
perSample = ['de_movingAve', 'de_LDS', 'psd_movingAve', 'psd_LDS']
array = np.zeros(shape=(len(directories),len(os.listdir(directories[0])), n, 4, 62, 5, 64)) # features = 4*62*5*64 (zero padded) // trials = (3 sessions) x 15 people x 24 labels 
li = []
for h, dire in enumerate(directories):
    print(dire)
    data = [loadmat(dire + file) for file in os.listdir(dire)]
    for i, bigsample in enumerate(data):
        for j in range(n):
            for k, key in enumerate(perSample):
                sample = np.transpose(np.array(bigsample[key + str(j+1)]), (0,2,1))
                #print(sample.shape)
                sample = np.pad(sample, [(0,0), (0,0), (0, 64-sample.shape[2])])
                #print(sample.shape)
                array[h][i][j][k] = sample

print(array.shape)



./data/eeg_feature_smooth/1/
./data/eeg_feature_smooth/2/
./data/eeg_feature_smooth/3/
(3, 15, 24, 4, 62, 5, 64)


In [5]:
_X = array.reshape(np.prod(array.shape[0:3]), *array.shape[3:])

In [6]:
_X.shape

(1080, 4, 62, 5, 64)

In [7]:
session1_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]
session2_label = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]
session3_label = [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]
labels = {0: 'neutral', 1: 'sad', 2: 'fear', 3: 'happy'}

y = np.array(session1_label * 15 + session2_label * 15 + session3_label * 15)
y.shape

(1080,)

In [8]:
X = _X.transpose(0, 4, 1, 2,3)
print(X.shape)
X = X.reshape(X.shape[0], X.shape[1], np.prod(X.shape[2:]))
print(X.shape)

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

(1080, 64, 4, 62, 5)
(1080, 64, 1240)


In [9]:
def fully_conv():
    return tf.keras.Sequential([
    tf.keras.layers.Conv1D(
128, 3, data_format='channels_last', padding="same", activation='elu',input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.Conv1D(
128, 3, data_format='channels_last', padding="same", activation='elu',input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.Conv1D(
128, 3, data_format='channels_last', padding="same", activation='elu',input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.Conv1D(
128, 3, data_format='channels_last', padding="same", activation='elu',input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [52]:
def single_conv():
    return tf.keras.Sequential([
    tf.keras.layers.Conv1D(
32, 3, activation='relu',input_shape=X.shape[1:]),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [58]:
def dense():
    return tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=X.shape[1:]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(4)
])

In [10]:
model = fully_conv()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 64, 128)           476288    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 32, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 32, 128)           49280     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 16, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 16, 128)           49280     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 8, 128)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 8, 128)            4

In [11]:
model.fit(X_train, y_train,epochs=30)

Train on 864 samples
Epoch 1/30
864/864 [==============================] - 9s 10ms/sample - loss: 89274830.5185 - accuracy: 0.2512
Epoch 2/30
864/864 [==============================] - 3s 4ms/sample - loss: 18615162.0729 - accuracy: 0.3056
Epoch 3/30
864/864 [==============================] - 3s 4ms/sample - loss: 2942563.5700 - accuracy: 0.3762
Epoch 4/30
864/864 [==============================] - 3s 4ms/sample - loss: 4837033.5615 - accuracy: 0.3125
Epoch 5/30
864/864 [==============================] - 3s 4ms/sample - loss: 8163885.8895 - accuracy: 0.3264
Epoch 6/30
864/864 [==============================] - 3s 4ms/sample - loss: 3525689.0739 - accuracy: 0.2917
Epoch 7/30
864/864 [==============================] - 3s 4ms/sample - loss: 64795668.7729 - accuracy: 0.3102
Epoch 8/30
864/864 [==============================] - 3s 4ms/sample - loss: 4502404.3906 - accuracy: 0.3611
Epoch 9/30
864/864 [==============================] - 3s 4ms/sample - loss: 11551491.4693 - accuracy: 0.2975
Ep

In [12]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

216/1 - 0s - loss: 14577868.5267 - accuracy: 0.4491

Test accuracy: 0.44907406
